In [246]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import scipy.signal as signal
import altair as alt

from pathlib import Path
from operator import and_
from functools import reduce
from itertools import chain, groupby, product, accumulate
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder

alt.data_transformers.enable('default', max_rows=None)
%load_ext autotime

DataTransformerRegistry.enable('default')

## Read Data

In [182]:
cols_names = [
    'acceleration, chest (X axis)',
    'acceleration, chest (Y axis)',
    'acceleration, chest (Z axis)',
    'ECG-1',
    'ECG-2',
    'acceleration, left-ankle (X axis)',
    'acceleration, left-ankle (Y axis)',
    'acceleration, left-ankle (Z axis)',
    'gyro, left-ankle (X axis)',
    'gyro, left-ankle (Y axis)',
    'gyro, left-ankle (Z axis)',
    'magnetometer, left-ankle (X axis)',
    'magnetometer, left-ankle (Y axis)',
    'magnetometer, left-ankle (Z axis)',
    'acceleration, right-lower-arm (X axis)',
    'acceleration, right-lower-arm (Y axis)',
    'acceleration, right-lower-arm (Z axis)',
    'gyro, right-lower-arm (X axis)',
    'gyro, right-lower-arm (Y axis)',
    'gyro, right-lower-arm (Z axis)',
    'magnetometer, right-lower-arm (X axis)',
    'magnetometer, right-lower-arm (Y axis)',
    'magnetometer, right-lower-arm (Z axis)',
    'activity'
]

In [183]:
activities_names = np.array([
    'N/A',
    'Standing still',
    'Sitting and relaxing',
    'Lying down',
    'Walking',
    'Climbing stairs',
    'Waist bends forward',
    'Frontal elevation of arms',
    'Knees bending (crouching)',
    'Cycling',
    'Jogging',
    'Running',
    'Jump front & back',
])

In [184]:
fs = 50

In [185]:
partial_dfs = []
for i in range(1, 11):
    partial_df = pd.read_table(Path('data')/ ('mHealth_subject' + str(i) + '.log'), header=None, names=cols_names)
    partial_df['timepoint'] =  np.arange(1/fs*len(partial_df), step=1/fs)[:len(partial_df)]
    partial_df['subject'] = i
    partial_dfs.append(partial_df)
    
df = pd.concat(partial_dfs)

## Data Exploration

In [186]:
df.head()

,"acceleration, chest (X axis)","acceleration, chest (Y axis)","acceleration, chest (Z axis)",ECG-1,ECG-2,"acceleration, left-ankle (X axis)","acceleration, left-ankle (Y axis)","acceleration, left-ankle (Z axis)","gyro, left-ankle (X axis)","gyro, left-ankle (Y axis)",...,"acceleration, right-lower-arm (Z axis)","gyro, right-lower-arm (X axis)","gyro, right-lower-arm (Y axis)","gyro, right-lower-arm (Z axis)","magnetometer, right-lower-arm (X axis)","magnetometer, right-lower-arm (Y axis)","magnetometer, right-lower-arm (Z axis)",activity,timepoint,subject
0,-9.8184,0.009971,0.29563,0.004186,0.004186,2.1849,-9.6967,0.63077,0.103900,-0.84053,...,0.187760,-0.44902,-1.0103,0.034483,-2.35000,-1.610200,-0.030899,0,0.00,1
1,-9.8489,0.524040,0.37348,0.004186,0.016745,2.3876,-9.5080,0.68389,0.085343,-0.83865,...,0.023595,-0.44902,-1.0103,0.034483,-2.16320,-0.882540,0.326570,0,0.02,1
2,-9.6602,0.181850,0.43742,0.016745,0.037677,2.4086,-9.5674,0.68113,0.085343,-0.83865,...,0.275720,-0.44902,-1.0103,0.034483,-1.61750,-0.165620,-0.030693,0,0.04,1
3,-9.6507,0.214220,0.24033,0.079540,0.117220,2.1814,-9.4301,0.55031,0.085343,-0.83865,...,0.367520,-0.45686,-1.0082,0.025862,-1.07710,0.006945,-0.382620,0,0.06,1
4,-9.7030,0.303890,0.31156,0.221870,0.205130,2.4173,-9.3889,0.71098,0.085343,-0.83865,...,0.407290,-0.45686,-1.0082,0.025862,-0.53684,0.175900,-1.095500,0,0.08,1


In [187]:
df['activity'] = activities_names[df['activity']]

In [188]:
def get_activity_intervals_df(df):
    activity_intervals = []
    for i in df.groupby('subject'):
        prev = None
        timepoints = []
        activities = []

        for ind, val in enumerate(i[1]['activity']):
            if val != prev:
                prev = val
                timepoints.append(df.iloc[ind]['timepoint'])
                activities.append(val)
        timepoints.append(i[1].iloc[-1]['timepoint'])

        activity_df = pd.DataFrame({
            "start": pd.to_datetime(timepoints[:-1], unit='s'), 
            "end": pd.to_datetime(timepoints[1:], unit='s'), 
            'activity': activities,
            'subject': i[0]
        })
        activity_df['length'] = (activity_df['end'] - activity_df['start'])/ np.timedelta64(1, 's') 
        activity_intervals.append(activity_df)
    return pd.concat(activity_intervals)

In [189]:
activity_length_df = get_activity_intervals_df(df)

In [190]:
alt.Chart(activity_length_df).mark_rule().encode(
    y = alt.Y('activity:N'),
    x = alt.X('start:T', axis=alt.Axis(title='time, mm:ss', format =('%M:%S'))),
    x2 = alt.X2('end:T')
).properties(
    width = 800/2,
    height= 300
).facet(
    column='subject:Q', 
    columns=2
)

alt.FacetChart(...)

We can see, that even though we nearly 1 hour of recording, we have nearly 12 minutes of usefull signals

In [191]:
alt.Chart(activity_length_df[activity_length_df['activity'] != activities_names[0]]).encode(
    x = 'length:Q',
    y = alt.Y('activity:N', title='length, s'),
).mark_bar().facet(column='subject:N', columns=5)

alt.FacetChart(...)

Jump front & back is much shorter, then previous ones (20s vs 1m), so I decided to drop it. Also, subject #7 has much shorter Climbing stairs activity (only 15 seconds), so we can drop it to use larger window size.

In [192]:
def get_signal(subject, activity):
    def get_interval():
        tmp_df = activity_length_df[activity_length_df['subject'] == subject]
        res = tmp_df[tmp_df['activity'] == activity]
        return res['start'].values[0], res['end'].values[0]
    
    st, end = get_interval()
    to_seconds = lambda x: x.astype('float')/10**9
    get_index = lambda x: np.argmin(np.abs(df['timepoint'].values - x))
    st, end = to_seconds(st), to_seconds(end)
    
    return df.iloc[get_index(st): get_index(end)]

## Data Preprocessing

In [193]:
window_length = 30
used_signals = ['ECG-1', 'ECG-2', 'timepoint']
used_signals += ['subject']
activity_to_signals = defaultdict(list)
# df
# activity_length_df
for activity in activities_names:
    if activity in ['N/A']:
        continue
    tmp_df = activity_length_df[activity_length_df['activity'] == activity]
    tmp_df = tmp_df[tmp_df['length'] > window_length]
    to_seconds = lambda x: x.astype('float')/10**9
    st = to_seconds(tmp_df['start'].values)
    end = to_seconds(tmp_df['end'].values)
    for st, en, subject in zip(st, end, tmp_df['subject'].values):
        length = (en-st) - ((en-st) % window_length)
        tmp_df = df[df['subject'] == subject]
        tmp_df = tmp_df[tmp_df['timepoint'] >= st]
        tmp_df = tmp_df[tmp_df['timepoint'] < st + length]
        
        if len(tmp_df) != 0:
            activity_to_signals[activity].append(tmp_df[used_signals])

In [194]:
signal_activity_pairs = []
for activity in activity_to_signals.keys():
    for signal_per_subject in activity_to_signals[activity]:
        for i in range(int(len(signal_per_subject)/(fs*window_length))):
            signal_activity_pairs.append((signal_per_subject.iloc[i*window_length*fs:(i+1)*window_length*fs], activity))

## Feature generating

In [195]:
def plot_ecg(sig,lead=1, rs=None):
    base = alt.Chart(sig).mark_line().encode(
        x = alt.X('timepoint:Q', axis=alt.Axis(labels=True), title='time, s'),
        y = alt.Y('ECG-'+str(lead)+':Q', title='Voltage, mV'),
    )
    
    if rs is None:
        return base
    
    return alt.layer(
        base,
        alt.Chart(pd.DataFrame({'label_time': rs})).mark_rule().encode(
            x = alt.X(field='label_time', type='quantitative', axis=alt.Axis(labels=False), title=''),
            color=alt.value('#ae1325')
        ),    
    )

def plot_series(sig, stem=False):
    df = pd.DataFrame(data=sig, index=sig.index).reset_index()
    base  = alt.Chart(df).encode(
        x = alt.X('timepoint:Q', axis=alt.Axis(labels=True), title='time, s', scale=alt.Scale(zero=False)),
        y = alt.Y('ECG-1:Q', title='Voltage, mV'),
    )
    if stem:
        return base.mark_rule() + base.mark_point()
    else:
        return base.mark_line()

In [196]:
#reduce(and_, [plot_ecg(get_signal(1, activity)).properties(title=activity) for activity in activities_names[1:]])

In [294]:
def get_rs(sig, show_step_visualization=False, lead=1):
    ecg_sig = sig['ECG-' + str(lead)]
    ecg_sig = ecg_sig.set_axis(sig['timepoint'], axis='index')

    d1 = ecg_sig.shift(1) - ecg_sig
    d2 = d1.shift(1) - d1
    d = d2**2
    d = d[d.notnull()]


    treshhold = max(d)*0.03
    picks = d[d > treshhold]

    after_threshold = picks.copy(deep=True)

    for i in picks.index:
        siblings = picks[np.abs(picks.index - i) < 0.075]
        if picks[i] != 0:
            for s in siblings.index:
                if s == i:
                    continue
                picks[s] = 0
    picks = picks[picks != 0]

    rs = []
    for qrs_region_center in picks.index:
        region = ecg_sig[np.abs(ecg_sig.index - qrs_region_center) < 0.075]
        mn = (min(region) + max(region))/2
        r = region[(region-mn) == (region - mn).max()].index[0]
        rs.append(r)
    
    if show_step_visualization:
        return plot_series(d).properties(title='Double Difference array') & \
        plot_series(after_threshold, stem=True).properties(title='Applied 3 % threshold') & \
        plot_series(picks, stem=True).properties(title='Remove in 75 ms neigbours') & \
        plot_ecg(sig, rs=rs).properties(title='With R labels')
    else:
        return np.array(rs)

In [199]:
def get_rr_intervals(r_locations):
    return r_locations[1:] - r_locations[:-1]

def get_nn50(rr_intervals):
    max_strike = 0
    cur_strike = 0
    prev_val = 0
    for i in rr_intervals:
        if i - prev_val < 0.05:
            cur_strike += 1
        else:
            if cur_strike > max_strike:
                max_strike = cur_strike
            cur_strike = 0
        prev_val = i
    if cur_strike > max_strike:
        max_strike = cur_strike
    return max_strike

def get_sssp_features(data, positive=False):
    res = [
        np.mean(data),
        np.median(data),
        np.std(data),
        stats.trim_mean(rr_intervals, 0.25),
        stats.skew(data),
        stats.kurtosis(data),
        np.max(data),
        np.min(data),
        stats.scoreatpercentile(data, 25),
        stats.scoreatpercentile(data, 75),
    ]
    if positive:
        res.extend([
            stats.gmean(data),
            stats.hmean(data),
            stats.gstd(data)
        ])
    return res


def get_time_domain_features(rr_intervals):
    # deep breathing difference
    dbd = max(rr_intervals) - min(rr_intervals)
    # the number of successive RR interval pairs differing by more than 50 ms
    nn50 = get_nn50(rr_intervals)
    # nn50 normalized by 
    pnn50 = get_nn50(rr_intervals)/len(rr_intervals)
    # root mean square of successive differences
    rmssd = np.sum((rr_intervals[1:] - rr_intervals[:-1])**2)
    return dbd, nn50, pnn50, rmssd

In [278]:
def get_hr_features(rr_intervals, show_visualization=False):
    timepoints = np.array(list(accumulate(rr_intervals)))
    hrv = np.interp(x=np.linspace(timepoints[0], timepoints[-1], 256), xp=timepoints, fp=rr_intervals)

    fourier = np.fft.fft(hrv)[:hrv.size//2]
    frequencies = np.fft.fftfreq(hrv.size, d=1/256)[:hrv.size//2]

    f, Pxx_den = signal.periodogram(hrv, 256)

    top_6_psd_from_hrv = list(map(lambda x: x[0], sorted(list(zip(f, Pxx_den)), key=lambda x: x[1], reverse=True)[:6]))

    if show_visualization:
        return (
            alt.Chart(pd.DataFrame({'rr_interval': hrv, 'timepoint': np.arange(0, 256, 1)})).encode(
                y = alt.Y('rr_interval:Q', title="RR interval duration"),
                x = alt.X('timepoint:Q', title="time, s")
            ).mark_line().properties(title='Heart rate variability') 
        ) | \
        (
            alt.Chart(pd.DataFrame({"frequency": f, "density": Pxx_den})).encode(
                x = alt.X('frequency:Q', title='Hz'),
                y = alt.Y('density:Q', title='density, U^2/Hz',)
            ).mark_line() + alt.Chart(pd.DataFrame({'top_frequency': top_6_psd_from_hrv, 'density': Pxx_den[list(map(int, top_6_psd_from_hrv))]})).encode(
                x = 'top_frequency:Q',
                y = 'density:Q'
            ).mark_point(color='red')
        ).properties(title='Top 6 power spectral density')
    return *top_6_psd_from_hrv, *get_sssp_features(hrv)

In [279]:
get_rs(sig = get_signal(1, 'Lying down')[:500], show_step_visualization=True)

alt.VConcatChart(...)

In [280]:
get_hr_features(rr_intervals, show_visualization=True)

alt.HConcatChart(...)

In [318]:
def get_rs(sig, show_step_visualization=False, lead=1):
    ecg_sig = sig['ECG-' + str(lead)]
    ecg_sig = ecg_sig.set_axis(sig['timepoint'], axis='index')

    d1 = ecg_sig.shift(1) - ecg_sig
    d2 = d1.shift(1) - d1
    d = d2**2
    d = d[d.notnull()]


    treshhold = max(d)*0.03
    picks = d[d > treshhold]

    after_threshold = picks.copy(deep=True)

    for i in picks.index:
        siblings = picks[np.abs(picks.index - i) < 0.075]
        if picks[i] != 0:
            for s in siblings.index:
                if s == i:
                    continue
                picks[s] = 0
    picks = picks[picks != 0]

    rs = []
    for qrs_region_center in picks.index:
        region = ecg_sig[np.abs(ecg_sig.index - qrs_region_center) < 0.075]
        mn = (min(region) + max(region))/2
        r = region[(region-mn) == (region - mn).max()].index[0]
        if not len(rs) or ((r - rs[-1]) != 0):
            rs.append(r)
    
    if show_step_visualization:
        return plot_series(d).properties(title='Double Difference array') & \
        plot_series(after_threshold, stem=True).properties(title='Applied 3 % threshold') & \
        plot_series(picks, stem=True).properties(title='Remove in 75 ms neigbours') & \
        plot_ecg(sig, rs=rs).properties(title='With R labels')
    else:
        return np.array(rs)

In [333]:
def get_ecg_features(sig, lead=1):
    rr_intervals = get_rr_intervals(get_rs(sig, lead=lead))
    # heart rate in beats per minute
    hr = 1/rr_intervals*60
    features = [
                *get_hr_features(rr_intervals),
                *get_sssp_features(rr_intervals), 
                *get_sssp_features(hr), 
                *get_time_domain_features(rr_intervals),
                ]
    return features

## Classification

In [371]:
X = []
y = []
for signals, activity in signal_activity_pairs:
    X.append(get_ecg_features(signals, lead=1))# + get_ecg_features(signals,lead=2))
    y.append(activity)
X = np.array(X)
y = np.array(y)

time: 25.5 s


(202, 40)

time: 2.47 ms


In [341]:
# len(signal_activity_pairs)
# %install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 891 µs


In [342]:
# get_sssp_features(sig['acceleration, chest (X axis)'], False)

time: 222 µs


In [326]:
# alt.Chart(pd.DataFrame({"ECG-1": signal.medfilt(signal_activity_pairs[0][0]['ECG-1']), 
#                         'timepoint': signal_activity_pairs[0][0]['timepoint']
#                                })).encode(
#     x = alt.X('timepoint:Q', axis=alt.Axis(labels=True), title='time, s', scale=alt.Scale(zero=False)),
#     y = alt.Y('ECG-1:Q', title='Voltage, mV'),
# ).mark_line()